# Preprocess the data

## Rigid registration and skull extraction

In [3]:
import os
from os.path import expanduser as eu
from headctools.registration import register_file_folder
from headctools.preprocessing import ct_skull

In [ ]:
import sys
import vtk

def stl_to_vtk(filename):
    # Read the .stl file 
    a = vtk.vtkSTLReader()
    a.SetFileName(filename)
    a.Update()
    a = a.GetOutput()

    # Write the .vtk file
    filename = filename.replace('.stl', '.vtk')
    b = vtk.vtkPolyDataWriter()
    b.SetFileName(filename)
    b.SetInputData(a)
    b.Update()
    return filename

In [ ]:
imgs = eu('~/Code/datasets/cq500/converted/selected/')
fixed_img = os.path.join(imgs, 'CQ500-CT-312_CT PRE CONTRAST THIN.nii.gz')
# Registered images will be saved in a sub folder with the name of the fixed image

params = {
    'reg_atlas_path': fixed_img
}

ct_skull(imgs, params = params)

## Mesh atlas construction

### Decimate meshes

In [ ]:
import vedo

print('Decimating meshes...')
decimate_percent = 0.005

meshes_folder = eu('~/Code/datasets/cq500/converted/selected/preprocessed_ct_to_skull/test')
print( f'  Input folder: {meshes_folder}')
meshes_paths = [os.path.join(meshes_folder, f) for f in os.listdir(meshes_folder) if f.endswith('.stl') and 'decimated' not in f]
saved_meshes = []
print(f'  Found {len(meshes_paths)} meshes')
for i, mesh in enumerate(meshes_paths):
    print(f'    [{i+1}/{len(meshes_paths)}] Input mesh: {os.path.split(mesh)[1]}')
    v_mesh = vedo.Mesh(mesh).decimate(decimate_percent)
    decimated_path = mesh.replace('.stl', f'_decimated_{int(decimate_percent*100)}perc_{v_mesh.points().shape[0]}points.stl')
    v_mesh.write(decimated_path)
    saved_meshes.append(decimated_path)
    print(f'      Decimated mesh saved in {decimated_path}')
    
saved_meshes = [stl_to_vtk(f) for f in saved_meshes]


In [ ]:
saved_meshes

## Deformetrica

In [ ]:
template_path = saved_meshes[0]
dataset_paths = saved_meshes[1:]

In [ ]:
import os
import deformetrica as dfca

iteration_status_dictionaries = []

def estimator_callback(status_dict):
    iteration_status_dictionaries.append(status_dict)
    return True

# instantiate a Deformetrica object
deformetrica = dfca.Deformetrica(output_dir='output', verbosity='INFO')

dataset_specifications = {
    'dataset_filenames': [[{'skull': f}] for f in dataset_paths],
    'subject_ids': [os.path.splitext(os.path.split(f)[1])[0] for f in dataset_paths],
}
template_specifications = {
    'skull': {'deformable_object_type': 'SurfaceMesh',
              'kernel_type': 'torch', 'kernel_width': 20.0,
              'noise_std': 10.0,
              'filename': template_path,
              'attachment_type': 'current'}
}
estimator_options={'optimization_method_type': 'GradientAscent', 'initial_step_size': 1.,
                   'max_iterations': 25, 'max_line_search_iterations': 10, 'callback': estimator_callback}

# perform a deterministic atlas estimation
model = deformetrica.estimate_deterministic_atlas(template_specifications, dataset_specifications,
                                                estimator_options=estimator_options,
                                                model_options={'deformation_kernel_type': 'torch', 'deformation_kernel_width': 40.0, 'dtype': 'float32'})

In [17]:
import vedo

print('Decimating meshes...')
decimate_percent = 0.005

meshes_folder = eu('~/Code/datasets/cq500/converted/selected/preprocessed_ct_to_skull/test')
print( f'  Input folder: {meshes_folder}')
meshes_paths = [os.path.join(meshes_folder, f) for f in os.listdir(meshes_folder) if f.endswith('.stl') and 'decimated' not in f]
saved_meshes = []
print(f'  Found {len(meshes_paths)} meshes')
for i, mesh in enumerate(meshes_paths):
    print(f'    [{i+1}/{len(meshes_paths)}] Input mesh: {os.path.split(mesh)[1]}')
    v_mesh = vedo.Mesh(mesh).decimate(decimate_percent)
    decimated_path = mesh.replace('.stl', f'_decimated_{int(decimate_percent*100)}perc_{v_mesh.points().shape[0]}points.stl')
    v_mesh.write(decimated_path)
    saved_meshes.append(decimated_path)
    print(f'      Decimated mesh saved in {decimated_path}')
    
saved_meshes = [stl_to_vtk(f) for f in saved_meshes]


Decimating meshes...
  Input folder: /home/fmatzkin/Code/datasets/cq500/converted/selected/preprocessed_ct_to_skull/test
  Found 4 meshes
    [1/4] Input mesh: CQ500-CT-54_CT Thin Plain.stl
      Decimated mesh saved in /home/fmatzkin/Code/datasets/cq500/converted/selected/preprocessed_ct_to_skull/test/CQ500-CT-54_CT Thin Plain_decimated_0perc_3312points.stl
    [2/4] Input mesh: CQ500-CT-182_CT PRE CONTRAST THIN.stl
      Decimated mesh saved in /home/fmatzkin/Code/datasets/cq500/converted/selected/preprocessed_ct_to_skull/test/CQ500-CT-182_CT PRE CONTRAST THIN_decimated_0perc_3507points.stl
    [3/4] Input mesh: CQ500-CT-405_CT Thin Plain.stl
      Decimated mesh saved in /home/fmatzkin/Code/datasets/cq500/converted/selected/preprocessed_ct_to_skull/test/CQ500-CT-405_CT Thin Plain_decimated_0perc_3425points.stl
    [4/4] Input mesh: CQ500-CT-133_CT Plain 3mm.stl
      Decimated mesh saved in /home/fmatzkin/Code/datasets/cq500/converted/selected/preprocessed_ct_to_skull/test/CQ500-CT-

In [18]:
saved_meshes

['/home/fmatzkin/Code/datasets/cq500/converted/selected/preprocessed_ct_to_skull/test/CQ500-CT-54_CT Thin Plain_decimated_0perc_3312points.vtk',
 '/home/fmatzkin/Code/datasets/cq500/converted/selected/preprocessed_ct_to_skull/test/CQ500-CT-182_CT PRE CONTRAST THIN_decimated_0perc_3507points.vtk',
 '/home/fmatzkin/Code/datasets/cq500/converted/selected/preprocessed_ct_to_skull/test/CQ500-CT-405_CT Thin Plain_decimated_0perc_3425points.vtk',
 '/home/fmatzkin/Code/datasets/cq500/converted/selected/preprocessed_ct_to_skull/test/CQ500-CT-133_CT Plain 3mm_decimated_0perc_3308points.vtk']

## Deformetrica

In [19]:
template_path = saved_meshes[0]
dataset_paths = saved_meshes[1:]

In [21]:
import os
import deformetrica as dfca

iteration_status_dictionaries = []

def estimator_callback(status_dict):
    iteration_status_dictionaries.append(status_dict)
    return True

# instantiate a Deformetrica object
deformetrica = dfca.Deformetrica(output_dir='output', verbosity='INFO')

dataset_specifications = {
    'dataset_filenames': [[{'skull': f}] for f in dataset_paths],
    'subject_ids': [os.path.splitext(os.path.split(f)[1])[0] for f in dataset_paths],
}
template_specifications = {
    'skull': {'deformable_object_type': 'SurfaceMesh',
              'kernel_type': 'torch', 'kernel_width': 20.0,
              'noise_std': 10.0,
              'filename': template_path,
              'attachment_type': 'current'}
}
estimator_options={'optimization_method_type': 'GradientAscent', 'initial_step_size': 1.,
                   'max_iterations': 25, 'max_line_search_iterations': 10, 'callback': estimator_callback}

# perform a deterministic atlas estimation
model = deformetrica.estimate_deterministic_atlas(template_specifications, dataset_specifications,
                                                estimator_options=estimator_options,
                                                model_options={'deformation_kernel_type': 'torch', 'deformation_kernel_width': 40.0, 'dtype': 'float32'})

Logger has been set to: INFO
>> No initial CP spacing given: using diffeo kernel width of 40.0
OMP_NUM_THREADS found in environment variables. Using value OMP_NUM_THREADS=4
context has already been set
>> No specified state-file. By default, Deformetrica state will by saved in file: output/deformetrica-state.p.
>> Set of 80 control points defined.
>> Momenta initialized to zero, for 3 subjects.
>> Started estimator: GradientAscent
------------------------------------- Iteration: 0 -------------------------------------
>> Log-likelihood = -1.675E+05 	 [ attachment = -1.675E+05 ; regularity = 0.000E+00 ]
>> Step size and gradient norm: 
		1.545E-05   and   6.471E+04 	[ landmark_points ]
		1.183E-04   and   8.457E+03 	[ momenta ]
------------------------------------- Iteration: 1 -------------------------------------
>> Log-likelihood = -1.589E+05 	 [ attachment = -1.589E+05 ; regularity = -3.070E+00 ]
>> Step size and gradient norm: 
		2.318E-05   and   6.003E+04 	[ landmark_points ]
		1